In [ ]:
import pygame
import random
import psycopg2
from datetime import datetime
import ctypes

# Дерекқормен байланыс орнату
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="snake_game_db",  # дерекқор аты
    user="aidarkyzydinara",
    password="20102010"
)
cur = conn.cursor()

# Пайдаланушы аты енгізу
username = input("Enter your username: ").strip()

# Пайдаланушы бар-жоғын тексеру
cur.execute("SELECT id FROM users WHERE username = %s", (username,))
user = cur.fetchone()

if user:
    # Егер бұрыннан бар болса, соңғы деңгейін аламыз
    user_id = user[0]
    cur.execute("SELECT MAX(level) FROM user_score WHERE user_id = %s", (user_id,))
    last_level = cur.fetchone()[0] or 1
    print(f"Welcome back, {username}! Your current level is: {last_level}")
else:
    # Егер жаңа пайдаланушы болса, оны дерекқорға қосамыз
    cur.execute("INSERT INTO users (username) VALUES (%s) RETURNING id", (username,))
    user_id = cur.fetchone()[0]
    conn.commit()
    last_level = 1  # жаңа қолданушыны 1 деңгейден бастаймыз
    print(f"Welcome, {username}! Let's start at level 1")

# Pygame бастапқы орнату
pygame.init()
font = pygame.font.SysFont("Arial", 25)
clock = pygame.time.Clock()

# Деңгейлерді орнату, жылдамдық пен қабырғалар
levels = {
    1: {"speed": 10, "walls": []},
    2: {"speed": 15, "walls": [(100,100,400,20), (100,300,20,200)]},
    3: {"speed": 20, "walls": [(150,150,20,300), (300,100,20,400)]}
}
# Қазір қандай деңгейде екенімізді тексереміз
level = last_level if last_level in levels else 1
speed = levels[level]["speed"]
walls = levels[level]["walls"]

# Ойын терезесін орнату
WIDTH, HEIGHT = 600, 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")



# Түстерді анықтау
WHITE = (255, 255, 255)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
BLACK = (0, 0, 0)

# Жыланның бастау жағдайы
snake = [(100, 100)]
dx, dy = 20, 0
food = (random.randint(0, 29)*20, random.randint(0, 29)*20)
score = 0
pause = False

# Экранды жаңарту үшін салатын функция
def draw():
    win.fill(WHITE)
    for x, y in snake:
        pygame.draw.rect(win, GREEN, (x, y, 20, 20))
    pygame.draw.rect(win, RED, (*food, 20, 20))
    for wx, wy, ww, wh in walls:
        pygame.draw.rect(win, BLACK, (wx, wy, ww, wh))
    text = font.render(f"Score: {score} | Level: {level}", True, BLACK)
    win.blit(text, (10, 10))
    pygame.display.update()

running = True
while running:
    pygame.time.delay(100)
    clock.tick(speed)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False  # Егер терезе жабылса, ойын тоқтайды

    keys = pygame.key.get_pressed()  # Басылған пернелерді аламыз

    # Жыланның қозғалысын басқару
    if keys[pygame.K_LEFT] and dx == 0:
        dx, dy = -20, 0
    elif keys[pygame.K_RIGHT] and dx == 0:
        dx, dy = 20, 0
    elif keys[pygame.K_UP] and dy == 0:
        dx, dy = 0, -20
    elif keys[pygame.K_DOWN] and dy == 0:
        dx, dy = 0, 20

    # Пауза жасау
    if keys[pygame.K_p]:
        pause = not pause

    # Ойыннан шығу
    elif keys[pygame.K_q]:
        running = False
        break

    if pause:
        continue  # Пауза болған кезде келесі қадамға өтпейміз

    head = (snake[0][0] + dx, snake[0][1] + dy)

    # Жылан қабырғаға соғысады немесе өзіне соғысады
    if head in snake or head[0] < 0 or head[0] >= WIDTH or head[1] < 0 or head[1] >= HEIGHT:
        break
    if any(wx <= head[0] < wx+ww and wy <= head[1] < wy+wh for wx, wy, ww, wh in walls):
        break

    snake.insert(0, head)  # Жыланның басын жаңа жерге қоямыз

    # Егер жеміс табылса, ұпай қосамыз және жаңа жеміс орнатамыз
    if head == food:
        score += 1
        food = (random.randint(0, 29)*20, random.randint(0, 29)*20)
    else:
        snake.pop()  # Жыланның соңғы бөлігін алып тастаймыз

    draw()  # Экранды жаңартып отырамыз

# Ойын біткен соң ұпайды дерекқорға сақтау
cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))
conn.commit()  # Өзгерістерді сақтаймыз
cur.close()  # Дерекқорды жабамыз
conn.close()  # Дерекқор байланысын жабамыз

# Pygame жабу
pygame.quit()

Enter your username:  gtggtg


Welcome, gtggtg! Let's start at level 1
